In [9]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [10]:
# Set the path to your dataset
PATH = 'my_dataset'
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'val')

In [11]:

# Get the number of files in each directory
total_train = sum([len(files) for r, d, files in os.walk(train_dir)])
total_val = sum([len(files) for r, d, files in os.walk(validation_dir)])


In [12]:
# Variables for pre-processing and training
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150


In [13]:
# Image Data Generators for augmenting the dataset
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


In [14]:
# Loading the training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary'
)

Found 550 images belonging to 2 classes.


In [15]:
validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='binary'
)

Found 70 images belonging to 2 classes.


In [138]:
# Step 2: - definig the cnn model

In [16]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])


/Users/rajjani/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
# Print model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
history = model.fit(train_generator, 
                    steps_per_epoch=total_train // batch_size,
                    validation_data=validation_generator,
                    validation_steps=total_val // batch_size,
                    epochs=epochs)

/Users/rajjani/opt/anaconda3/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.5111 - loss: 0.7830 - val_accuracy: 0.3429 - val_loss: 0.7211
Epoch 2/15
1/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.4062 - loss: 0.7297

2024-10-18 16:06:49.389260: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/rajjani/opt/anaconda3/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 230ms/step - accuracy: 0.4062 - loss: 0.7297 - val_accuracy: 0.6571 - val_loss: 0.6819
Epoch 3/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.6117 - loss: 0.6590 - val_accuracy: 0.6571 - val_loss: 0.6529
Epoch 4/15
1/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.6797 - loss: 0.6213

2024-10-18 16:07:07.289884: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 359ms/step - accuracy: 0.6484 - loss: 0.6149 - val_accuracy: 0.6571 - val_loss: 0.6488
Epoch 11/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.6812 - loss: 0.5998 - val_accuracy: 0.6571 - val_loss: 0.6307
Epoch 12/15
1/4 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.5789 - loss: 0.6462

2024-10-18 16:08:07.375357: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 365ms/step - accuracy: 0.5789 - loss: 0.6462 - val_accuracy: 0.6571 - val_loss: 0.6400
Epoch 13/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.6557 - loss: 0.5539 - val_accuracy: 0.6571 - val_loss: 0.6272
Epoch 14/15
1/4 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6053 - loss: 0.5620

2024-10-18 16:08:32.575942: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.6053 - loss: 0.5620 - val_accuracy: 0.6571 - val_loss: 0.6264
Epoch 15/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.6730 - loss: 0.5243 - val_accuracy: 0.6571 - val_loss: 0.6479


In [19]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")  # Convert to percentage

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 879ms/step - accuracy: 0.6571 - loss: 0.6479
Validation Accuracy: 65.71%


In [21]:
# Function to predict a single image
def predict_image(image_path):
    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    prediction = model.predict(img_array)

    # Set a confidence threshold
    threshold = 0.5
    if prediction[0][0] > threshold:
        return "Dog"
    elif prediction[0][0] < (1 - threshold):
        return "Cat"
    else:
        return "Unknown (not a cat or dog)"


In [24]:
# Example usage for an unknown image
image_path = 'dog.jpeg'  # Replace with the path to your input image
result = predict_image(image_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
The image is classified as: Dog
